# Jaeger-query with trace DSL

This notebook loads a trace from Jaeger query service via gRPC and runs analysis on it.

## Install library to the local maven repository
This step is only needed if trace DSL source code has been modified.
Open terminal in Jupyter and run the following command:
```
cd work && ./mvnw clean install -DskipTests
```

In [2]:
%%loadFromPOM
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java-util</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-protobuf</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-netty</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-stub</artifactId>
  <version>1.28.0</version>
</dependency>

<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
<dependency>
    <groupId>org.apache.tinkerpop</groupId>
    <artifactId>tinkergraph-gremlin</artifactId>
    <version>3.4.3</version>
</dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [9]:
%maven io.jaegertracing:jaeger-tracedsl:0.3.0
%maven io.jaegertracing:jaeger-proto:0.3.0

System.out.println(io.jaegertracing.analytics.gremlin.Keys.class);

class io.jaegertracing.analytics.gremlin.Keys


## Run analysis
It calculates maximum trace height and network latencies. It connects to Jaeger query service via gRPC.

In [38]:
import com.google.protobuf.ByteString;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;
import io.jaegertracing.analytics.NetworkLatency.Name;
import io.jaegertracing.analytics.gremlin.GraphCreator;
import io.jaegertracing.analytics.model.Converter;
import io.jaegertracing.analytics.model.Trace;
import io.jaegertracing.api_v2.Model.Span;
import io.jaegertracing.api_v2.Query.GetTraceRequest;
import io.jaegertracing.api_v2.Query.SpansResponseChunk;
import io.jaegertracing.api_v2.QueryServiceGrpc;
import io.jaegertracing.api_v2.QueryServiceGrpc.QueryServiceBlockingStub;
import java.util.ArrayList;
import java.util.Iterator;
import java.util.List;
import java.util.Map;
import java.util.Set;
import org.apache.tinkerpop.gremlin.structure.Graph;

String queryHostPort = "192.168.0.31:16686";
String traceIdStr = "5a8c9a8c69b0fd4f";

ManagedChannel channel;
channel = ManagedChannelBuilder.forTarget(queryHostPort).usePlaintext().build();
QueryServiceBlockingStub queryService = QueryServiceGrpc.newBlockingStub(channel);

ByteString traceId = Converter.toProtoId(traceIdStr);
Iterator<SpansResponseChunk> traceProto = queryService.getTrace(
    GetTraceRequest.newBuilder().setTraceId(traceId).build());

List<Span> spans = new ArrayList<>();
while (traceProto.hasNext()) {
  spans.addAll(traceProto.next().getSpansList());
}
Trace trace = Converter.toModel(spans);
Graph graph = GraphCreator.create(trace);

int height = TraceHeight.calculate(graph);
Map<Name, Set<Double>> networkLatencies = NetworkLatency.calculate(graph);

System.out.printf("Trace height = %d\n", height);
System.out.printf("Network latencies = %s\n", networkLatencies);

Trace height = 4
Network latencies = {Name{client='frontend', server='route'}=[2.05E-4, 6.64E-4, 2.0E-4, 1.56E-4, 1.82E-4, 1.99E-4, 6.42E-4, 1.55E-4, 5.54E-4, 5.77E-4], Name{client='frontend', server='customer'}=[6.04E-4], Name{client='frontend', server='driver'}=[4.66E-4]}


java.io.PrintStream@23aef5d1

## Write your own data analysis with Trace DSL and Apache Gremlin

In [31]:
// Are two spans connected?
TraceTraversal<Vertex, Vertex> traversal = graph.traversal(TraceTraversalSource.class)
        .hasName("HTTP GET: /customer")
        .repeat(__.out())
        .until(__.hasName("SQL SELECT"));

traversal.forEachRemaining(v -> Util.printVertex(v));
